In [ ]:
import pandas as pd
import os

In [ ]:
ppd_folder = "ppd/"
files = os.listdir(ppd_folder)

dfs = []
for f in files:
    tmp = pd.read_parquet(ppd_folder+f)
    dfs.append(tmp)
ppd = pd.concat(dfs, ignore_index=True)

In [ ]:
ppd

In [7]:
from datetime import date

In [1]:
import pandas as pd


csv_url = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv"



header = ['TRANSACTION_UNIQUE_IDENTIFIER','PRICE',
        'DATE_OF_TRANSFER','POSTCODE','PROPERTY_TYPE',
        'OLD_NEW','DURATION','PAON','SAON','STREET',
        'LOCALITY','TOWN_CITY','DISTRICT','COUNTY',
        'PPD_CATEGORY_TYPE','RECORD_STATUS_MONTHLY_FILE_ONLY']

chunk_size = 100

df_iter = pd.read_csv(csv_url, iterator=True, chunksize=chunk_size, header=None, names=header)


In [18]:
df_prova = next(df_iter)

In [19]:
df_prova.dtypes

TRANSACTION_UNIQUE_IDENTIFIER      object
PRICE                               int64
DATE_OF_TRANSFER                   object
POSTCODE                           object
PROPERTY_TYPE                      object
OLD_NEW                            object
DURATION                           object
PAON                               object
SAON                               object
STREET                             object
LOCALITY                           object
TOWN_CITY                          object
DISTRICT                           object
COUNTY                             object
PPD_CATEGORY_TYPE                  object
RECORD_STATUS_MONTHLY_FILE_ONLY    object
dtype: object

In [21]:
df_prova[
    (df_prova['COUNTY'] == 'GREATER LONDON')
    &(df_prova["DATE_OF_TRANSFER"]>'1995-11-01')
]

,TRANSACTION_UNIQUE_IDENTIFIER,PRICE,DATE_OF_TRANSFER,POSTCODE,PROPERTY_TYPE,OLD_NEW,DURATION,PAON,SAON,STREET,LOCALITY,TOWN_CITY,DISTRICT,COUNTY,PPD_CATEGORY_TYPE,RECORD_STATUS_MONTHLY_FILE_ONLY
110,{3639489A-69D0-4872-A19D-91F63E5CA203},200500,1995-11-20 00:00,SE23 2QA,D,N,F,40,NaN,SUNDERLAND ROAD,LONDON,LONDON,LEWISHAM,GREATER LONDON,A,A
159,{5B71326C-4F5F-48DC-BC21-91F6E807D94B},87950,1995-11-30 00:00,SE18 4PR,F,N,L,ROBERTSON HOUSE,FLAT 5,ROYAL HERBERT PAVILIONS,LONDON,LONDON,GREENWICH,GREATER LONDON,A,A


In [35]:
def transform(df):    
    df = df[
        (df['COUNTY'] == 'GREATER LONDON')
        &(df["DATE_OF_TRANSFER"]>'1995-11-01')
    ]
    return df

In [36]:
df_transformed = transform(df_prova)

In [39]:
len(df_transformed)

2

,TRANSACTION_UNIQUE_IDENTIFIER,PRICE,DATE_OF_TRANSFER,POSTCODE,PROPERTY_TYPE,OLD_NEW,DURATION,PAON,SAON,STREET,LOCALITY,TOWN_CITY,DISTRICT,COUNTY,PPD_CATEGORY_TYPE,RECORD_STATUS_MONTHLY_FILE_ONLY
0,{5BBE9CB3-6332-4EB0-9CD3-8737CEA4A65A},42000,1995-12-21 00:00,NE4 9DN,S,N,F,8,NaN,MATFEN PLACE,FENHAM,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A
8,{F7A5CAE4-BDD3-4911-897C-8AC393B97281},43000,1995-12-01 00:00,LA6 3DQ,T,N,F,90,NaN,NEW VILLAGE,INGLETON,CARNFORTH,CRAVEN,NORTH YORKSHIRE,A,A
75,{5D4952FA-5C76-405F-8F82-8738E6E28A4E},54000,1995-12-20 00:00,MK18 3QZ,T,N,F,7,NaN,RUDDS CLOSE,WINSLOW,BUCKINGHAM,AYLESBURY VALE,BUCKINGHAMSHIRE,A,A
92,{B07FDFED-3E8D-4BAF-A933-8739608D7590},60000,1995-12-18 00:00,SM5 3DY,S,N,F,21,NaN,PARK LANE,CARSHALTON,CARSHALTON,SUTTON,GREATER LONDON,A,A


In [4]:
df_prova.to_parquet("notrans.parquet")

In [50]:
df_prova['DATE_OF_TRANSFER'] = pd.to_datetime(df_prova['DATE_OF_TRANSFER'])

In [52]:
transformed = df_prova[df_prova['DATE_OF_TRANSFER'].dt.date >= date(1995,5,1)]

In [59]:
transformed['DATE_OF_TRANSFER'] = transformed['DATE_OF_TRANSFER'].astype(str)

/tmp/ipykernel_122997/2999378639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transformed['DATE_OF_TRANSFER'] = transformed['DATE_OF_TRANSFER'].astype(str)


In [44]:
transformed = df_prova[df_prova['DATE_OF_TRANSFER'] >= '1995-11-01']

In [60]:
transformed.to_parquet("trans.parquet")

In [46]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [47]:
session = SparkSession.builder.getOrCreate()

In [61]:
df = session.read.parquet("trans.parquet")

In [62]:
df.show()

+-----------------------------+------+-------------------+--------+-------------+-------+--------+------------+--------+-------------------+----------------+---------------+--------------------+------------------+-----------------+-------------------------------+-----------------+
|TRANSACTION_UNIQUE_IDENTIFIER| PRICE|   DATE_OF_TRANSFER|POSTCODE|PROPERTY_TYPE|OLD_NEW|DURATION|        PAON|    SAON|             STREET|        LOCALITY|      TOWN_CITY|            DISTRICT|            COUNTY|PPD_CATEGORY_TYPE|RECORD_STATUS_MONTHLY_FILE_ONLY|__index_level_0__|
+-----------------------------+------+-------------------+--------+-------------+-------+--------+------------+--------+-------------------+----------------+---------------+--------------------+------------------+-----------------+-------------------------------+-----------------+
|         {611A9774-3EFE-4C...| 50000|1995-06-30 00:00:00| WF8 2EY|            S|      N|       F|          12|    NULL|        PRAIL CLOSE|      PONTEFRA

In [ ]:
ppd_folder = "ppd/"
files = os.listdir(ppd_folder)

In [ ]:
df = session.read.parquet(ppd_folder+files[0])

In [48]:
# import pandas as pd
# import os
# import oci

# config = oci.config.from_file("/config/oci/config", "DEFAULT")
# object_storage_client = oci.object_storage.ObjectStorageClient(config)

In [49]:
OCI_ACCESS_KEY_ID = os.environ['OCI_ACCESS_KEY_ID']
OCI_SECRET_ACCESS_KEY = os.environ['OCI_SECRET_ACCESS_KEY']

In [ ]:
if len(chunk)>0:
    chunk.to_parquet(local_file_path)
    bucket_file_path = f"{bucket_folder}{file_name}"
    with open(local_file_path, 'rb') as file:
        object_storage_client.put_object(namespace, bucket_name, bucket_file_path, file)
        print('transfer')
    os.remove(local_file_path)

In [51]:
import boto3

ModuleNotFoundError: No module named 'boto3'

In [52]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 49.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 11.1 MB/s eta 0:00:00
